In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
%matplotlib inline

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

import glob 
import os 

In [2]:
def create_labels_dict(label_file):
    dict = {}
    start_idx = 0
    with open(label_file, "r") as f:
        for line in f:
            dict[line.strip()] = start_idx
            start_idx += 1
    return dict

In [3]:
def prepare_ds(data_dir, label_dict):
    files = glob.glob(f"{data_dir}/**/*.csv")
    train_data = []
    train_labels = []
    for file in files:
        parent_dir = os.path.basename(os.path.dirname(file))
        csv_data = pd.read_csv(file)
        csv_data = csv_data.drop(columns=["timestamp"])
        n = csv_data.shape[0]
        indices = np.linspace(0, n-1, num=100, dtype=int)
        new_csv_data = csv_data.iloc[indices]
        train_data.append(new_csv_data)
        train_labels.append(label_dict[parent_dir])

    return train_data, train_labels

In [4]:
import torch.nn.functional as F

label_file = "labels.txt"
label_dict = create_labels_dict(label_file)


train_dir = "data/train"
train_data, train_labels = prepare_ds(train_dir, label_dict)
train_data = np.array(train_data)
train_data = np.vstack(train_data)
train_labels = np.array(train_labels)


val_dir = "data/val"
val_data, val_labels = prepare_ds(val_dir, label_dict)
val_data = np.array(val_data)
val_data = np.vstack(val_data)
val_labels = np.array(val_labels)


scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)

train_data = train_data.reshape(-1, 100, 6)
val_data = val_data.reshape(-1, 100, 6)

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

class HandActivitiesDataset(Dataset):
    def __init__(self, dataDir, labels_dict, transform=None):
        self.dataDir = dataDir
        self.label_dict = labels_dict
        self.transform = transform
        self.data = []
        self.labels = []
        self.load_data()
        

    def load_data(self):
        files = glob.glob(f"{self.dataDir}/**/*.csv")
        for file in files:
            parent_dir = os.path.basename(os.path.dirname(file))
            csv_data = pd.read_csv(file)
            csv_data = csv_data.drop(columns=["timestamp"])
            n = csv_data.shape[0]
            indices = np.linspace(0, n-1, num=100, dtype=int)
            new_csv_data = csv_data.iloc[indices]
            self.data.append(new_csv_data)
            self.labels.append(self.label_dict[parent_dir])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

In [6]:
train_X = torch.tensor(train_data).float()
train_y = torch.tensor(train_labels)
train_y = F.one_hot(train_y, num_classes=3)
val_X = torch.tensor(val_data).float()
val_y = torch.tensor(val_labels)
val_y = F.one_hot(val_y, num_classes=3)

train_ds = TensorDataset(train_X, train_y)
val_ds = TensorDataset(val_X, val_y)

train_ds = DataLoader(train_ds, batch_size=32, shuffle=True)
val_ds = DataLoader(val_ds, batch_size=16)


In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, seq_len, dropout=0.5, output_size=3 ):
        super(RNN, self).__init__()
        
        # LSTM Layers
        self.lstm_1 = nn.LSTM(input_size, hidden_sizes[0], num_layers=2,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_21 = nn.LSTM(2*hidden_sizes[0], hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_22 = nn.LSTM(input_size, hidden_sizes[1], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_31 = nn.LSTM(2*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_32 = nn.LSTM(4*hidden_sizes[1], hidden_sizes[2], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_41 = nn.LSTM(2*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        self.lstm_42 = nn.LSTM(4*hidden_sizes[2], hidden_sizes[3], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        hidd = 2*hidden_sizes[0] + 4*(hidden_sizes[1]+hidden_sizes[2]+hidden_sizes[3])
        self.lstm_5 = nn.LSTM(hidd, hidden_sizes[4], num_layers=2,
                             batch_first=True, bidirectional=True, dropout=dropout)
        
        # Fully Connected Layer
        self.fc = nn.Sequential(nn.Linear(2*hidden_sizes[4]*seq_len, 4096),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(4096, 1024),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=dropout),
                                nn.Linear(1024, output_size),
                                nn.Sigmoid()
                               )
        
    def forward(self, x):
        # lstm layers:
        x1, _ = self.lstm_1(x)
        
        x_x1, _ = self.lstm_21(x1)
        x_x2, _ = self.lstm_22(x)
        x2 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_31(x_x1)
        x_x2, _ = self.lstm_32(x2)
        x3 = torch.cat([x_x1, x_x2], dim=2)
        
        x_x1, _ = self.lstm_41(x_x1)
        x_x2, _ = self.lstm_42(x3)
        x4 = torch.cat([x_x1, x_x2], dim=2)
        x = torch.cat([x1, x2, x3, x4], dim=2)
        x, _ = self.lstm_5(x)
        
        # fully connected layers:
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [8]:
### VALIDATION FUNCTION
def validation(model, loader, criterion, device="cpu"):
    model.eval()
    loss = 0
    preds_all = torch.FloatTensor()
    labels_all = torch.FloatTensor()
    
    with torch.no_grad():
        for batch_x, labels in loader:
            labels_all = torch.cat((labels_all, labels), dim=0)
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            output = model.forward(batch_x)
            loss += criterion(output,labels).item()
            preds_all = torch.cat((preds_all, output.to("cpu")), dim=0)
    total_loss = loss/len(loader)
    auc_score = roc_auc_score(labels_all, preds_all)
    return total_loss, auc_score

In [9]:
def train_model(model, trainloader, validloader, criterion, optimizer, 
                scheduler, epochs=20, device="cpu", print_every=1):
    model.to(device)
    best_auc = 0
    best_epoch = 0
    for e in range(epochs):
        model.train()
        
        for batch_x, labels in trainloader:
            batch_x, labels = batch_x.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            
            # Training 
            optimizer.zero_grad()
            output = model.forward(batch_x)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
        # at the end of each epoch calculate loss and auc score:
        model.eval()
        train_loss, train_auc = validation(model, trainloader, criterion, device)
        valid_loss, valid_auc = validation(model, validloader, criterion, device)
        if valid_auc > best_auc:
            best_auc = valid_auc
            best_epoch = e
            torch.save(model.state_dict(), "best-state.pt")
        if e % print_every == 0:
            to_print = "Epoch: "+str(e+1)+" of "+str(epochs)
            to_print += ".. Train Loss: {:.4f}".format(train_loss)
            to_print += ".. Valid Loss: {:.4f}".format(valid_loss)
            to_print += ".. Valid AUC: {:.3f}".format(valid_auc)
            print(to_print)
    # After Training:
    model.load_state_dict(torch.load("best-state.pt"))
    to_print = "\nTraining completed. Best state dict is loaded.\n"
    to_print += "Best Valid AUC is: {:.4f} after {} epochs".format(best_auc,best_epoch+1)
    print(to_print)
    return

In [10]:
### PREDICTION FUNCTION
def prediction(model, loader, device="cpu"):
    model.to(device)
    model.eval()
    preds_all = torch.FloatTensor()
    
    with torch.no_grad():
        for batch_x in loader:
            batch_x = batch_x.to(device)
            
            output = model.forward(batch_x).to("cpu")
            preds_all = torch.cat((preds_all, output), dim=0)
    return preds_all

In [11]:
hidden_sizes = [288, 192, 144, 96, 32]
max_learning_rate = 0.001
epochs = 41
input_size = 6
sequence_length = 100
# Model
print(input_size, hidden_sizes, sequence_length)
model_lstm = RNN(input_size, hidden_sizes, sequence_length)
print("Model: ")
print(model_lstm)

# criterion, optimizer, scheduler
#criterion = nn.MSELoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=max_learning_rate)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                          max_lr = max_learning_rate,
                                          epochs = epochs,
                                          steps_per_epoch = len(train_ds),
                                          pct_start = 0.2,
                                          anneal_strategy = "cos")


6 [288, 192, 144, 96, 32] 100
Model: 
RNN(
  (lstm_1): LSTM(6, 288, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_21): LSTM(576, 192, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_22): LSTM(6, 192, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_31): LSTM(384, 144, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_32): LSTM(768, 144, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_41): LSTM(288, 96, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_42): LSTM(576, 96, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (lstm_5): LSTM(2304, 32, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=1024, bias=True)
    (4

In [ ]:
# Checking if GPU is available
if torch.cuda.is_available():
    my_device = "cuda"
    print("GPU is enabled")
else:
    my_device = "cpu"
    print("No GPU :(")

train_losses = []
val_losses = []
model_lstm.to(my_device)
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    model_lstm.train()
    for inputs, labels in train_ds:
        inputs, labels = inputs.to(my_device).float(), labels.to(my_device).float()
        # print(labels.shape)
        # labels = labels.unsqueeze(1).float()
        # print(labels.shape)

        optimizer.zero_grad()
        output = model_lstm(inputs)
        # output = output.unsqueeze(1)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss/len(train_ds))
    
    model_lstm.eval()
    preds = []
    labels_all = []
    with torch.no_grad():
        for inputs, labels in val_ds:
            inputs, labels = inputs.to(my_device).float(), labels.to(my_device).float()
            output = model_lstm(inputs)
            pred = torch.argmax(output.to("cpu"), axis=1)
            label = torch.argmax(labels.to("cpu"))
            # print(label)
            # print(pred)
            preds.append(pred)
            labels_all.append(label)
            loss = criterion(output.squeeze(), labels.float())
            val_loss += loss.item()
        val_losses.append(val_loss/len(val_ds))
    

    # print(preds)
    # print(labels_all)
    correct = sum(1 for a, b in zip(preds, labels_all) if a == b)
    total = len(preds)
    acc = correct / total * 100
        # auc_score = roc_auc_score(labels_all, preds_all)
    print("Epoch: {}/{}.. ".format(epoch+1, epochs),
          "Training Loss: {:.3f}.. ".format(train_losses[-1]),
          "Validation Loss: {:.3f}.. ".format(val_losses[-1]),
          "Validation AUC: {:.3f}%".format(acc))

GPU is enabled
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([16, 3])
torch.Size([6, 3])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [30]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    my_device = "cuda"
    print("GPU is enabled")
else:
    my_device = "cpu"
    print("No GPU :(")

train_losses = []
val_losses = []

# Move model to the appropriate device
model_lstm.to(my_device)

for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    model_lstm.train()

    # Training loop
    for inputs, labels in train_ds:
        inputs, labels = inputs.to(my_device).float(), labels.to(my_device).float()

        optimizer.zero_grad()
        output = model_lstm(inputs)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_losses.append(train_loss / len(train_ds))

    # Validation loop
    model_lstm.eval()
    preds = []
    labels_all = []
    
    with torch.no_grad():
        for inputs, labels in val_ds:
            inputs, labels = inputs.to(my_device).float(), labels.to(my_device).float()
            output = model_lstm(inputs)

            # Get predicted and true labels
            pred = torch.argmax(output, dim=1).cpu()  # Convert to CPU
            label = torch.argmax(labels, dim=1).cpu()  # Convert to CPU

            preds.append(pred)
            labels_all.append(label)

            loss = criterion(output.squeeze(), labels.float())
            val_loss += loss.item()

    val_losses.append(val_loss / len(val_ds))

    # Convert lists to tensors for accuracy calculation
    preds = torch.cat(preds)
    labels_all = torch.cat(labels_all)

    correct = torch.eq(preds, labels_all).sum().item()
    total = labels_all.size(0)
    acc = (correct / total) * 100  # Convert to percentage

    # Print results
    print("Epoch: {}/{}.. ".format(epoch + 1, epochs),
          "Training Loss: {:.3f}.. ".format(train_losses[-1]),
          "Validation Loss: {:.3f}.. ".format(val_losses[-1]),
          "Validation Accuracy: {:.2f}%".format(acc))

GPU is enabled
Epoch: 1/41..  Training Loss: 0.000..  Validation Loss: 0.028..  Validation Accuracy: 93.33%
Epoch: 2/41..  Training Loss: 0.000..  Validation Loss: 0.027..  Validation Accuracy: 94.00%
Epoch: 3/41..  Training Loss: 0.000..  Validation Loss: 0.027..  Validation Accuracy: 94.67%
Epoch: 4/41..  Training Loss: 0.000..  Validation Loss: 0.027..  Validation Accuracy: 94.67%
Epoch: 5/41..  Training Loss: 0.000..  Validation Loss: 0.026..  Validation Accuracy: 94.67%
Epoch: 6/41..  Training Loss: 0.000..  Validation Loss: 0.026..  Validation Accuracy: 94.67%
Epoch: 7/41..  Training Loss: 0.000..  Validation Loss: 0.026..  Validation Accuracy: 95.33%
Epoch: 8/41..  Training Loss: 0.000..  Validation Loss: 0.025..  Validation Accuracy: 95.33%
Epoch: 9/41..  Training Loss: 0.000..  Validation Loss: 0.025..  Validation Accuracy: 95.33%
Epoch: 10/41..  Training Loss: 0.000..  Validation Loss: 0.025..  Validation Accuracy: 95.33%
Epoch: 11/41..  Training Loss: 0.000..  Validation Los